```
```

### backtrader 를 이용한 백테스팅 ###
---------------------------------------------------

!pip install backtrader
backtrader         1.9.76.123

In [1]:
import mariadb
import pandas   as pd
from datetime import datetime

>> 종목 리스트 가져오기

In [3]:
try:
    conn = mariadb.connect(
        user="root",
        password="123",
        host="localhost",
        port=3306,
        database="home"
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")

cursor = conn.cursor()
cursor.execute("select * from daily_price where  CODE = ?",[('100840')])
data = cursor.fetchall()
desc = list(map(lambda x : x[0],cursor.description))
conn.close()

df =  pd.DataFrame(data=data, columns=desc)
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='raise')
df = df.set_index(df['date'],drop=True)
df = df.drop(columns=['code','date','chg'])
df.head(5)

,open,high,low,close,volume
date,,,,,
2022-01-03,18850,18900,18600,18800,8954
2022-01-04,18850,18850,18600,18600,8138
2022-01-05,18750,18750,18450,18450,7794
2022-01-06,18200,18500,18000,18050,12756
2022-01-07,18050,18550,18050,18450,8534


[backtrader docu]https://www.backtrader.com/docu/

In [4]:
import backtrader as bt

In [7]:
class MyStrategy(bt.Strategy) :
    params = (
        ('period',20),
        ('devfactor',2),
        ('debug',False)
    )
    def log(self, txt, dt=None) :
        dt = dt or self.data.datetime[0]
        if isinstance(dt, float) :
            df = bt.num2date(dt)
        print(f'{df.isoformat()}, {txt}')

    def  __init__(self) :
        self.bol1 = bt.ind.BollingerBands(period=self.p.period,devfactor=self.p.devfactor, plot=True)
    def next(self) :
        global size
        if not self.position :
            if self.data.low[0] < self.bol1.lines.bot[0] :
                bottom = self.bol1.lines.bot[0]
                size = int(self.broker.getcash() /bottom)
                self.buy(price=bottom, size = size)
                self.log(f'BUY CREATE {bottom:.2f}')
                print(size, 'EA')
        else :
            if self.data.high[0] < self.bol1.lines.mid[0] :
                self.sell(price=self.bol1.lines.mid[0], size=size)
                self.log(f'SELL CREATE {self.bol1.lines.mid[0]}')
                print(size, 'EA')

cerebro = bt.Cerebro()
cerebro.addstrategy(MyStrategy)
data = bt.feeds.PandasData(dataname=df)
cerebro.adddata(data)
cerebro.broker.setcash(20000000)
cerebro.broker.setcommission(0.00015)
print(f'Starting Portfolio Value: {cerebro.broker.getvalue():.2f}')
cerebro.run()
print(f'Final Portfolio Value: {cerebro.broker.getvalue():.2f}')
cerebro.plot(style='candle', barup='red', bardown='blue', volumn=False)[0][0]


Starting Portfolio Value: 20000000.00
2022-01-28T00:00:00, BUY CREATE 16337.79
1224 EA
2022-04-07T00:00:00, BUY CREATE 18369.11
1088 EA
2022-04-08T00:00:00, SELL CREATE 18950.0
1088 EA
2022-04-11T00:00:00, BUY CREATE 18215.87
1094 EA
2022-04-12T00:00:00, SELL CREATE 18875.0
1094 EA
2022-04-28T00:00:00, BUY CREATE 17717.79
1121 EA
2022-06-09T00:00:00, BUY CREATE 18414.27
1079 EA
2022-06-10T00:00:00, BUY CREATE 18316.85
1085 EA
2022-06-13T00:00:00, SELL CREATE 18927.5
1085 EA
2022-06-14T00:00:00, BUY CREATE 17904.69
1091 EA
2022-06-15T00:00:00, SELL CREATE 18745.0
1091 EA
2022-06-16T00:00:00, BUY CREATE 17028.12
1122 EA
2022-06-17T00:00:00, BUY CREATE 16697.22
1144 EA
2022-06-20T00:00:00, SELL CREATE 18317.5
1144 EA
2022-06-21T00:00:00, BUY CREATE 15873.00
1138 EA
2022-06-22T00:00:00, SELL CREATE 17927.5
1138 EA
2022-06-23T00:00:00, BUY CREATE 14778.35
1180 EA
2022-09-26T00:00:00, BUY CREATE 17339.95
1005 EA
2022-09-27T00:00:00, SELL CREATE 21040.0
1005 EA
2023-01-02T00:00:00, BUY CREATE

AttributeError: 'RRuleLocator' object has no attribute 'set_view_interval'